# Экспериментальная часть

In [418]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import string
from scipy.spatial.distance import jensenshannon
import scipy.io as sio
from scipy.special import rel_entr, kl_div
from scipy.stats import entropy, ks_2samp
from scipy.spatial.distance import jensenshannon
from skmultiflow.drift_detection import DDM, PageHinkley, ADWIN
from skmultiflow.data import ConceptDriftStream
import datetime
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
import os

In [181]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear', C = 2.8, penalty = 'l1')
samples_number = 7

In [3]:
df_all = pd.read_csv('../Amazon_Electronics/Electronics_5.csv')
df_all

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"
...,...,...,...,...,...,...,...,...,...
1689183,A34BZM6S9L7QI4,B00LGQ6HL8,"Candy Cane ""Is it just me?""","[1, 1]",Burned these in before listening to them for a...,5,Boom -- Pop -- Pow. These deliver.,1405555200,"07 17, 2014"
1689184,A1G650TTTHEAL5,B00LGQ6HL8,"Charles Spanky ""Zumina Reviews""","[0, 0]",Some people like DJ style headphones or earbud...,5,"Thin and light, without compromising on sound ...",1405382400,"07 15, 2014"
1689185,A25C2M3QF9G7OQ,B00LGQ6HL8,Comdet,"[0, 0]",I&#8217;m a big fan of the Brainwavz S1 (actua...,5,Same form factor and durability as the S1 with...,1405555200,"07 17, 2014"
1689186,A1E1LEVQ9VQNK,B00LGQ6HL8,J. Chambers,"[0, 0]","I've used theBrainwavz S1 In Ear Headphones, a...",5,Superb audio quality in a very comfortable set...,1405641600,"07 18, 2014"


In [397]:
df_all.reviewTime = pd.to_datetime(df_all.reviewTime, format="%m %d, %Y")
df_all = df_all.sort_values(by = 'reviewTime').reset_index(drop = True)
df_all["bin_label"] = df_all.overall > 3
df_all["label"] = df_all.overall.apply(lambda x: '3' if x == 3 else ('1-2' if x < 3 else '4-5'))
df_all["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), df_all.reviewText.apply(lambda x: np.str_(x)))))
df_all

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,bin_label,words,label
0,A1ABVP0DV1ZN89,B00000JBAT,Adam Sacks,"[0, 0]",The RIO rocks! It is so great that Diamond Mul...,5,WOW! THE BEST THING TO HAPPEN TO MUSIC!,929232000,1999-06-13,True,the rio rock it is so great that diamond multi...,4-5
1,A6ILK3FXYH595,B00000J4FS,kc2kth,"[1, 1]",I have found a few places that will do memory ...,5,After-market Memory Upgrades,931132800,1999-07-05,True,have found few place that will do memori upgra...,4-5
2,A7RV1KU5O0II9,B00000JFMK,"David S. Saunders ""Equal parts tech geek, bra...","[0, 0]",I love this player! I travel frequently and th...,5,Incredible screen!,931392000,1999-07-08,True,love thi player travel frequent and the l50 ke...,4-5
3,AJ6TMOHHFJJAJ,B00000JMO4,Ali Navaie,"[5, 5]",If you are looking for the best camcorder in t...,5,Simply the best in the market --- Get it while...,932688000,1999-07-23,True,if you are look for the best camcord in the ma...,4-5
4,A1JTSRG8SU4VFO,B00000JFE3,L. Goff,"[0, 0]",The &quot;lip-synch&quot; problem has shown up...,5,Own the Sony 550D model,932688000,1999-07-23,True,the quot lip synch quot problem ha shown up on...,4-5
...,...,...,...,...,...,...,...,...,...,...,...,...
1689183,A3CW0ZLUO5X2B1,B00JOS04PK,"35-year Technology Consumer ""8-tracks to 802.11""","[1, 1]",...between an iPod Nano (7th generation) and w...,5,Does what it's supposed to: moves data and pow...,1406073600,2014-07-23,True,between an ipod nano 7th gener and whatev othe...,4-5
1689184,A3HYQHTMEQI1MI,B00GAKX34E,Tim Williams,"[0, 0]",It installed easy just had to use the adapter ...,4,Fits in my Cooler Master HAF 912,1406073600,2014-07-23,True,it instal easi just had to use the adapt that ...,4-5
1689185,A2IFGGXG3YV3Y6,B00KMRGF28,"Scott Anderson ""Sharpsburg""","[0, 0]",This is a fully functional laptop at under $30...,4,High Quality Basics!,1406073600,2014-07-23,True,thi is fulli function laptop at under 300 00 t...,4-5
1689186,A1QCZQTJNK3MU,B00J46VVKE,"Bakeman ""no this isnt me...""","[0, 0]","was cheap, works what else can you say about a...",5,cheap and functional,1406073600,2014-07-23,True,wa cheap work what els can you say about charg...,4-5


In [398]:
df_by_years = {}
for year in df_all.reviewTime.dt.year.unique().tolist():
    df_by_years[year] = df_all[df_all.reviewTime.dt.year == year].reset_index(drop = True)
    print(year, len(df_by_years[year]))
    print(df_by_years[year].bin_label.value_counts(), '\n')

1999 72
True     61
False    11
Name: bin_label, dtype: int64 

2000 817
True     686
False    131
Name: bin_label, dtype: int64 

2001 1609
True     1292
False     317
Name: bin_label, dtype: int64 

2002 2315
True     1802
False     513
Name: bin_label, dtype: int64 

2003 3547
True     2663
False     884
Name: bin_label, dtype: int64 

2004 5159
True     3655
False    1504
Name: bin_label, dtype: int64 

2005 9638
True     6994
False    2644
Name: bin_label, dtype: int64 

2006 15447
True     11528
False     3919
Name: bin_label, dtype: int64 

2007 35976
True     28404
False     7572
Name: bin_label, dtype: int64 

2008 49872
True     39460
False    10412
Name: bin_label, dtype: int64 

2009 70666
True     55304
False    15362
Name: bin_label, dtype: int64 

2010 103797
True     80475
False    23322
Name: bin_label, dtype: int64 

2011 173395
True     134860
False     38535
Name: bin_label, dtype: int64 

2012 282942
True     224559
False     58383
Name: bin_label, dtype: int64 

2

In [458]:
skf = StratifiedKFold(n_splits=samples_number)
data_by_year_splitted = {}
for year in range(2007, 2015):
    data_tmp = df_by_years[year].sample(35000)
    data_list, labels_list = [], []
    
    for train_index, test_index in skf.split(data_tmp.words, data_tmp.label):
        data = data_tmp.iloc[test_index]
        data_list.append(data.words)
        labels_list.append(data.label)
        
    data_by_year_splitted[year] = {'data': data_list, 'labels': labels_list}
    print(year)

2007
2008
2009
2010
2011
2012
2013
2014


In [459]:
df_accuracy = pd.DataFrame(np.zeros([2015-2007, (2015-2007)*samples_number]), index=range(2007, 2015), columns=pd.MultiIndex.from_product([range(2007, 2015), range(samples_number)]))
df_tvd = df_accuracy.copy()
df_js = df_accuracy.copy()
df_ks = df_accuracy.copy()
df_perc_no_zero = df_accuracy.copy()
df_accuracy

2007                               2008            ... 2013            \
        0    1    2    3    4    5    6    0    1    2  ...    4    5    6   
2007  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2008  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2009  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2010  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2011  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2012  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2013  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2014  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     2014                                
        0    1    2    3    4    5    6  
2007  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2008  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2009  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2010  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2011  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2012  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2013  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2014  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[8 rows x 56 columns]

In [460]:
for year_train in range(2007, 2015):
    for test_index in range(samples_number):
        vecotrizer = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
        clf = LogisticRegression(solver='liblinear', C = 2.8, penalty = 'l1', class_weight='balanced')
        train_data = pd.Series(dtype=pd.StringDtype())
        train_labels = pd.Series(dtype=int)
        for train_index in range(samples_number):
            if test_index != train_index:
                train_data = train_data.append(data_by_year_splitted[year_train]['data'][train_index])
                train_labels = train_labels.append(data_by_year_splitted[year_train]['labels'][train_index])
        train_data = vecotrizer.fit_transform(train_data)
        clf.fit(train_data, train_labels)
        for year_test in range(year_train, 2015):
            test_data = vecotrizer.transform(data_by_year_splitted[year_test]['data'][test_index])
            test_labels = data_by_year_splitted[year_test]['labels'][test_index]
            df_accuracy.loc[year_train][year_test][test_index] = accuracy_score(clf.predict(test_data),test_labels)
            
            dist_train = np.array(train_data.mean(axis=0))[0]
            dist_test = np.array(test_data.mean(axis=0))[0]
            tvd = sum(abs(dist_train - dist_test))/2
            js = jensenshannon(dist_train, dist_test)
            ks = ks_2samp(dist_train, dist_test).statistic
            perc_no_zero = np.count_nonzero(dist_test)/np.count_nonzero(dist_train)
            df_tvd.loc[year_train][year_test][test_index] = tvd
            df_js.loc[year_train][year_test][test_index] = js
            df_ks.loc[year_train][year_test][test_index] = ks
            df_perc_no_zero.loc[year_train][year_test][test_index] = perc_no_zero
            print("Train in {0}, test in {1}, test sample {2}\naccuracy_score = {3}, TVD = {4}, JS = {5}, KS = {6}".format(year_train, year_test, test_index, df_accuracy.loc[year_train][year_test][test_index], tvd, js, ks))
            
# df_accuracy = df_accuracy/samples_number
# df_accuracy[df_accuracy.eq(0)] = np.nan

Train in 2007, test in 2007, test sample 0
accuracy_score = 0.8344, TVD = 1.539732619282053, JS = 0.16151290599923718, KS = 0.2204579387308057
Train in 2007, test in 2008, test sample 0
accuracy_score = 0.8342, TVD = 1.626466228740188, JS = 0.1646831830633494, KS = 0.2208502708181214
Train in 2007, test in 2009, test sample 0
accuracy_score = 0.8374, TVD = 1.750362819775341, JS = 0.16864670274079505, KS = 0.22014189343824586
Train in 2007, test in 2010, test sample 0
accuracy_score = 0.8306, TVD = 1.897556476078634, JS = 0.18309441258692502, KS = 0.23961682232805503
Train in 2007, test in 2011, test sample 0
accuracy_score = 0.836, TVD = 2.0789126788099668, JS = 0.20019794737225163, KS = 0.261260475811637
Train in 2007, test in 2012, test sample 0
accuracy_score = 0.844, TVD = 2.248576278849461, JS = 0.2225893786956777, KS = 0.3108359942893885
Train in 2007, test in 2013, test sample 0
accuracy_score = 0.8464, TVD = 2.5912611169246285, JS = 0.25552728942142094, KS = 0.3743719962074565


Train in 2008, test in 2010, test sample 0
accuracy_score = 0.8342, TVD = 1.8035833117958666, JS = 0.1774098771784459, KS = 0.24487577955187195
Train in 2008, test in 2011, test sample 0
accuracy_score = 0.8304, TVD = 1.9919116999146276, JS = 0.19353923141257706, KS = 0.26429601641375666
Train in 2008, test in 2012, test sample 0
accuracy_score = 0.8422, TVD = 2.224786581095448, JS = 0.21664329823112685, KS = 0.3134154021167246
Train in 2008, test in 2013, test sample 0
accuracy_score = 0.8434, TVD = 2.635468174411265, JS = 0.2514467319867226, KS = 0.3795274950738416
Train in 2008, test in 2014, test sample 0
accuracy_score = 0.8616, TVD = 2.723786257177906, JS = 0.25838607056128804, KS = 0.3924269201860768
Train in 2008, test in 2008, test sample 1
accuracy_score = 0.8408, TVD = 1.6005963731941375, JS = 0.16069950797014004, KS = 0.2176258441545173
Train in 2008, test in 2009, test sample 1
accuracy_score = 0.8364, TVD = 1.6594032614911356, JS = 0.1654497675089754, KS = 0.2244097303813

Train in 2009, test in 2013, test sample 1
accuracy_score = 0.8486, TVD = 2.6416940368864155, JS = 0.24648603627918922, KS = 0.3763934553802455
Train in 2009, test in 2014, test sample 1
accuracy_score = 0.8512, TVD = 2.7703120893239332, JS = 0.25637723192743345, KS = 0.3927647322036536
Train in 2009, test in 2009, test sample 2
accuracy_score = 0.8352, TVD = 1.6254468303155174, JS = 0.16172887616672724, KS = 0.21984098195115734
Train in 2009, test in 2010, test sample 2
accuracy_score = 0.8264, TVD = 1.706926122339278, JS = 0.1693702464697033, KS = 0.23285065832687915
Train in 2009, test in 2011, test sample 2
accuracy_score = 0.8316, TVD = 1.9049495138864914, JS = 0.18634297899869393, KS = 0.2586641307438163
Train in 2009, test in 2012, test sample 2
accuracy_score = 0.8468, TVD = 2.2126532720741263, JS = 0.21335035234960556, KS = 0.3118400799992157
Train in 2009, test in 2013, test sample 2
accuracy_score = 0.8532, TVD = 2.662017949560942, JS = 0.2482194509480462, KS = 0.38137861393

Train in 2010, test in 2011, test sample 5
accuracy_score = 0.8392, TVD = 1.7183599171277077, JS = 0.17370784173244178, KS = 0.24234688729694281
Train in 2010, test in 2012, test sample 5
accuracy_score = 0.842, TVD = 1.9996156468272919, JS = 0.19679387051965372, KS = 0.2875390979719503
Train in 2010, test in 2013, test sample 5
accuracy_score = 0.8462, TVD = 2.462312243802121, JS = 0.2325537828320772, KS = 0.3606497830693169
Train in 2010, test in 2014, test sample 5
accuracy_score = 0.8518, TVD = 2.584067243949748, JS = 0.2425394690816603, KS = 0.3769548985975179
Train in 2010, test in 2010, test sample 6
accuracy_score = 0.8264, TVD = 1.590706304494857, JS = 0.16318370933806844, KS = 0.2254926692431107
Train in 2010, test in 2011, test sample 6
accuracy_score = 0.8336, TVD = 1.6923080246404034, JS = 0.17335293011108557, KS = 0.24409957317134698
Train in 2010, test in 2012, test sample 6
accuracy_score = 0.845, TVD = 2.035857394073054, JS = 0.20069460432105576, KS = 0.295520821771288

Train in 2013, test in 2013, test sample 0
accuracy_score = 0.8486, TVD = 1.2284955142440126, JS = 0.16317838662326636, KS = 0.21344487913930416
Train in 2013, test in 2014, test sample 0
accuracy_score = 0.865, TVD = 1.2562760292648374, JS = 0.16922485211067878, KS = 0.22546108704764564
Train in 2013, test in 2013, test sample 1
accuracy_score = 0.8498, TVD = 1.2286264161380818, JS = 0.16232842531409736, KS = 0.20963153025780665
Train in 2013, test in 2014, test sample 1
accuracy_score = 0.855, TVD = 1.2772276140707932, JS = 0.1715443842392305, KS = 0.2328812553455168
Train in 2013, test in 2013, test sample 2
accuracy_score = 0.8448, TVD = 1.2214472712014075, JS = 0.1640230171026857, KS = 0.21697356490200181
Train in 2013, test in 2014, test sample 2
accuracy_score = 0.849, TVD = 1.2705868820087451, JS = 0.17077228632905458, KS = 0.2323010392690242
Train in 2013, test in 2013, test sample 3
accuracy_score = 0.8468, TVD = 1.2220553247254244, JS = 0.16294370138437347, KS = 0.2137327027

In [461]:
df_accuracy_mean = pd.DataFrame(np.zeros([2015-2007, 2015-2007]), index=range(2007, 2015), columns=range(2007, 2015))

for year_train in range(2007, 2015):
    for year_test in range(year_train, 2015):
        df_accuracy_mean.loc[year_train][year_test] = df_accuracy.loc[year_train][year_test].mean()

df_accuracy_mean[df_accuracy_mean.eq(0)] = np.nan
ur_style = (df_accuracy_mean
            .style
            .background_gradient(cmap='Greens', axis=0)
            .highlight_null('white')
            .format("{:.2%}", na_rep=""))
ur_style

C:\Users\User\anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,2007,2008,2009,2010,2011,2012,2013,2014
2007,83.82%,84.17%,83.55%,83.05%,83.45%,84.40%,84.61%,85.02%
2008,,83.93%,83.68%,83.09%,83.43%,84.42%,84.85%,85.25%
2009,,,83.76%,82.85%,83.37%,84.46%,84.79%,85.15%
2010,,,,82.95%,83.59%,84.33%,85.03%,85.43%
2011,,,,,83.66%,84.45%,85.25%,85.68%
2012,,,,,,84.73%,85.20%,85.35%
2013,,,,,,,84.90%,85.47%
2014,,,,,,,,85.35%


In [462]:
df_tvd_mean = pd.DataFrame(np.zeros([2015-2007, 2015-2007]), index=range(2007, 2015), columns=range(2007, 2015))

for year_train in range(2007, 2015):
    for year_test in range(year_train, 2015):
        df_tvd_mean.loc[year_train][year_test] = df_tvd.loc[year_train][year_test].mean()

df_tvd_mean[df_tvd_mean.eq(0)] = np.nan
ur_style = (df_tvd_mean
            .style
            .background_gradient(cmap='Greens', axis=0)
            .highlight_null('white')
            .format("{:.3}", na_rep=""))
ur_style

C:\Users\User\anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,2007,2008,2009,2010,2011,2012,2013,2014
2007,1.54,1.63,1.76,1.91,2.07,2.27,2.59,2.69
2008,,1.59,1.67,1.81,1.99,2.24,2.63,2.74
2009,,,1.62,1.71,1.9,2.21,2.64,2.76
2010,,,,1.59,1.71,2.01,2.45,2.57
2011,,,,,1.54,1.73,2.18,2.3
2012,,,,,,1.41,1.69,1.8
2013,,,,,,,1.22,1.26
2014,,,,,,,,1.19


In [463]:
df_js_mean = pd.DataFrame(np.zeros([2015-2007, 2015-2007]), index=range(2007, 2015), columns=range(2007, 2015))

for year_train in range(2007, 2015):
    for year_test in range(year_train, 2015):
        df_js_mean.loc[year_train][year_test] = df_js.loc[year_train][year_test].mean()

df_js_mean[df_js_mean.eq(0)] = np.nan
ur_style = (df_js_mean
            .style
            .background_gradient(cmap='Greens', axis=0)
            .highlight_null('white')
            .format("{:.3}", na_rep=""))
ur_style

C:\Users\User\anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,2007,2008,2009,2010,2011,2012,2013,2014
2007,0.163,0.164,0.17,0.183,0.2,0.224,0.255,0.264
2008,,0.163,0.165,0.177,0.194,0.219,0.251,0.26
2009,,,0.163,0.171,0.187,0.213,0.247,0.256
2010,,,,0.162,0.174,0.198,0.232,0.242
2011,,,,,0.162,0.181,0.215,0.225
2012,,,,,,0.163,0.192,0.201
2013,,,,,,,0.163,0.17
2014,,,,,,,,0.164


In [464]:
df_ks_mean = pd.DataFrame(np.zeros([2015-2007, 2015-2007]), index=range(2007, 2015), columns=range(2007, 2015))

for year_train in range(2007, 2015):
    for year_test in range(year_train, 2015):
        df_ks_mean.loc[year_train][year_test] = df_ks.loc[year_train][year_test].mean()

df_ks_mean[df_ks_mean.eq(0)] = np.nan
ur_style = (df_ks_mean
            .style
            .background_gradient(cmap='Greens', axis=0)
            .highlight_null('white')
            .format("{:.3}", na_rep=""))
ur_style

C:\Users\User\anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,2007,2008,2009,2010,2011,2012,2013,2014
2007,0.223,0.218,0.22,0.237,0.263,0.311,0.375,0.391
2008,,0.224,0.223,0.241,0.265,0.314,0.379,0.394
2009,,,0.224,0.238,0.261,0.311,0.377,0.392
2010,,,,0.223,0.243,0.292,0.359,0.375
2011,,,,,0.222,0.267,0.335,0.351
2012,,,,,,0.22,0.286,0.302
2013,,,,,,,0.215,0.23
2014,,,,,,,,0.219


In [465]:
df_perc_no_zero_mean = pd.DataFrame(np.zeros([2015-2007, 2015-2007]), index=range(2007, 2015), columns=range(2007, 2015))

for year_train in range(2007, 2015):
    for year_test in range(year_train, 2015):
        df_perc_no_zero_mean.loc[year_train][year_test] = df_perc_no_zero.loc[year_train][year_test].mean()

df_perc_no_zero_mean[df_perc_no_zero_mean.eq(0)] = np.nan
ur_style = (df_perc_no_zero_mean
            .style
            .background_gradient(cmap='Greens', axis=0)
            .highlight_null('white')
            .format("{:.3}", na_rep=""))
ur_style

C:\Users\User\anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,2007,2008,2009,2010,2011,2012,2013,2014
2007,0.91,0.914,0.913,0.9,0.877,0.835,0.76,0.74
2008,,0.91,0.912,0.899,0.877,0.833,0.757,0.736
2009,,,0.91,0.899,0.878,0.834,0.756,0.736
2010,,,,0.911,0.894,0.853,0.777,0.756
2011,,,,,0.911,0.876,0.805,0.785
2012,,,,,,0.91,0.85,0.832
2013,,,,,,,0.911,0.898
2014,,,,,,,,0.909
